In [1]:
import pickle
import xml.etree.ElementTree as ET

import pandas as pd

In [2]:
with open("./cs_arxiv.pickle", "rb") as f:
    e = pickle.load(f)

In [3]:
data = {
    "id": [],
    "created": [],
    "updated": [],
    "authors": [],
    "title": [],
    "categories": [],
    "comments": [],
    "msc_class": [],
    "abstract": []
}

padding = "{http://arxiv.org/OAI/arXiv/}"
for a in e:
    for el in a[2]:
        try:
            elem = el.find("{http://www.openarchives.org/OAI/2.0/}metadata").find(padding+"arXiv")
        except:
            continue
        data["id"].append(elem.find(padding+"id").text)

        data["created"].append(elem.find(padding+"created").text)

        # Not every article has been updated
        try:
            data["updated"].append(elem.find(padding+"updated").text)
        except:
            data["updated"].append("")

        data["authors"].append([])
        for i in elem.find(padding+"authors"):
            if i.find(padding+"forenames") is not None:
                forenames = " "+i.find(padding+"forenames").text
            else:
                forenames = ""
            if i.find(padding+"affiliation") is not None:
                affiliation = ":"+i.find(padding+"affiliation").text
            else:
                affiliation = ""
            keyname = i.find(padding+"keyname").text
            data["authors"][-1].append(keyname+forenames+affiliation)

        data["title"].append(elem.find(padding+"title").text)

        data["categories"].append(elem.find(padding+"categories").text)


        if elem.find(padding+"comments") is not None:
            data["comments"].append(elem.find(padding+"comments").text)
        else:
            data["comments"].append("")

        if elem.find(padding+"msc-class") is not None:
            data["msc_class"].append(elem.find(padding+"msc-class").text)
        else:
            data["msc_class"].append("")

        data["abstract"].append(elem.find(padding+"abstract").text)

In [4]:
df = pd.DataFrame(data)
df.to_csv("../data/full_arxiv_cs_raw.csv", encoding="utf8")

df['primary_category'] = df.categories.str.split(expand=True).iloc[:,0]
df.loc[:,'abstract'] = df.abstract.str.strip().replace("\n", " ", regex=True)
df.applymap(lambda x: None if x=="" else x)

df.to_csv("../data/full_arxiv_cs_clean.csv", encoding="utf8")